# Проект по SQL

### Описание проекта. Его название. Цели.Описание датасета.

Цель проекта -проанализировать базу данных,в ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Задачи текущего проекта следующие:

- Знакомство с каждым файлом
- Посчитаем, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитаем количество обзоров и среднюю оценку;
- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — для исключиния из анализа брошюры;
- Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

Как мы будем решать текущие задачи:

- Исследуем таблицы — выведите первые строки;
- Сделаем по одному SQL-запросу для решения каждого задания;
- Выведем результаты каждого запроса в тетрадке;
- Сделаем выводы по каждой из решённых задач.

 **Описание данных**
 
  **Таблица books**
 
 
Содержит данные о книгах:


book_id — идентификатор книги;


author_id — идентификатор автора;


title — название книги;


num_pages — количество страниц;


publication_date — дата публикации книги;


publisher_id — идентификатор издателя.




 **Таблица authors**


Содержит данные об авторах:


author_id — идентификатор автора;


author — имя автора.





 **Таблица publishers**


Содержит данные об издательствах:


publisher_id — идентификатор издательства;


publisher — название издательства;





 **Таблица ratings**


Содержит данные о пользовательских оценках книг:


rating_id — идентификатор оценки;


book_id — идентификатор книги;


username — имя пользователя, оставившего оценку;


rating — оценка книги.





 **Таблица reviews**


Содержит данные о пользовательских обзорах:


review_id — идентификатор обзора;


book_id — идентификатор книги;


username — имя автора обзора;


text — текст обзора.

### Загрузка датасетов. И всех необходимых библиотек.

Воспользуемся инструкцией для доступа к базе данных:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Теперь посмотрим каждый датасет.

  **Таблица books**

In [2]:
query = '''SELECT *
FROM books
LIMIT 5'''

pd.io.sql.read_sql(query, con = engine)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


 **Таблица authors**

In [3]:
query = '''SELECT *
FROM authors
LIMIT 5'''

pd.io.sql.read_sql(query, con = engine)


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


 **Таблица publishers**

In [4]:
query = '''SELECT *
FROM publishers
LIMIT 5'''

pd.io.sql.read_sql(query, con = engine)


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


 **Таблица ratings**

In [5]:
query = '''SELECT *
FROM ratings
LIMIT 5'''

pd.io.sql.read_sql(query, con = engine)


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


 **Таблица reviews**

In [6]:
query = '''SELECT *
FROM reviews
LIMIT 5'''

pd.io.sql.read_sql(query, con = engine)


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Переходим к перечню заданий.

1.Посчитайте, сколько книг вышло после 1 января 2000 года;

In [7]:
query = '''SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(query, con = engine)


,count
0,819


Итак добавилось целых 819 книг.

2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [8]:
query = '''SELECT b.book_id, 
       b.title, 
       COUNT(DISTINCT rv.review_id) AS qty_reviews, 
       ROUND(AVG(ra.rating),3) AS mark_rating
FROM books AS b
LEFT JOIN reviews AS rv
ON b.book_id = rv.book_id
LEFT JOIN  ratings AS ra
ON b.book_id = ra.book_id
GROUP BY b.book_id, b.title
ORDER BY qty_reviews DESC'''

pd.io.sql.read_sql(query, con = engine)


,book_id,title,qty_reviews,mark_rating
0,948,Twilight (Twilight #1),7,3.663
1,963,Water for Elephants,6,3.977
2,734,The Glass Castle,6,4.207
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667
996,808,The Natural Way to Draw,0,3.000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000
998,221,Essential Tales and Poems,0,4.000


Итак, у нас получилось 1000 книг ровно. Для каких-то книг сделали 7 обзоров, а для каких-то вообще нет. Оценка рейтинга гуляет, причем не видно на первый взгляд зависимости от наличия обзоров. Так у книги The Cat in the Hat and Other Dr. Seuss Favorites рейтинг 5, при отсутствии обзоров, а у книги Twilight (Twilight #1) рейтинг 3,66 при максимальных 7 обзорах. Вывод, сколько обзоров не делай, на рейтинг это не повляет, можно только привлечь пользователей для прочтения, а вот выводы они уже сами сделают. И оценка уже зависит только от автора книги и личных предпочтений читателя.

3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [9]:
query = '''SELECT b.publisher_id, p.publisher, COUNT(*) count_books
            FROM books b
            LEFT JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE num_pages >= 50
            GROUP BY b.publisher_id, p.publisher
            ORDER BY COUNT(*) DESC
            LIMIT 3'''
pd.io.sql.read_sql(query, con = engine)


,publisher_id,publisher,count_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25


Итак, больше всего книг выпущено издательством Penguin Books	 - 42 книги. Хороший результат. Следующий Vintage с 31 книгой и замыкает тройку Grand Central Publishing с 25 книгами.

4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [10]:
query = '''WITH 
real_books AS (
SELECT book_id
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) > 50)


SELECT a.author_id, 
       a.author as имя_автора, 
       AVG(r.rating) AS его_рейтинг
FROM books AS b

INNER JOIN real_books
ON b.book_id = real_books.book_id

INNER JOIN authors AS a
ON b.author_id = a.author_id

INNER JOIN ratings AS r
ON real_books.book_id = r.book_id

GROUP BY a.author_id, a.author
ORDER BY AVG(r.rating) DESC

LIMIT 5 '''
pd.io.sql.read_sql(query, con = engine)

,author_id,имя_автора,его_рейтинг
0,236,J.K. Rowling/Mary GrandPré,4.287097
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.246914
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645


Самая популярная это J.K. Rowling/Mary GrandPré с рейтингом 4,28.  Не сильно отстал Markus Zusak/Cao Xuân Việt Khương с рейтингом 4,26. И очень рядом мой любимый Джон Рональд Руэл Толкин с рейтингом 4,24. 

5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
query = '''SELECT AVG (rev_count) as среднее_количество_обзоров
  FROM ( SELECT username,
        COUNT(review_id) as rev_count
        FROM reviews
        GROUP BY username
        HAVING username IN (
                            SELECT username
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT (rating_id) > 48)) as reviews'''
pd.io.sql.read_sql(query, con = engine)


,среднее_количество_обзоров
0,24.0


Пользователи, которые поставили больше 48 оценок в среднем пишут 24 обзора.

Выведите таблицу, которая будет содержать по году публикации:

- количество издательств,
- выпущенных книг и
- сколько всего тысяч страниц было в изданных книгах

(отобразить только те года, в которых издано более 30 книг)

In [12]:
query = '''SELECT EXTRACT(YEAR FROM b.publication_date) AS год_публикации,
COUNT(DISTINCT b.publisher_id) AS количество_издательств,
COUNT(b.book_id) AS количество_книг,
round(SUM(b.num_pages)::NUMERIC/1000,2) AS страниц_1000
FROM books AS b
GROUP BY год_публикации
HAVING COUNT(b.book_id) > 30
ORDER BY год_публикации
'''
pd.io.sql.read_sql(query, con = engine)

,год_публикации,количество_издательств,количество_книг,страниц_1000
0,1999.0,26,41,15.76
1,2000.0,35,38,13.33
2,2001.0,41,60,21.76
3,2002.0,62,94,38.60
4,2003.0,65,105,41.42
5,2004.0,88,124,46.78
6,2005.0,89,139,55.97
7,2006.0,109,184,68.30
8,2007.0,38,50,18.26


Выведите в одной таблице два числа — среднюю оценку тех книг, на которые написало отзывов более 3 человек и отдельно среднюю оценку остальных книг, сделайте выводы какой рейтинг больше

In [13]:
query = '''SELECT AVG(CASE WHEN(SELECT COUNT(*)
FROM ratings
WHERE book_id = r.book_id)>3
THEN r.rating END) AS средняя_оценка,
                  AVG(CASE WHEN(SELECT COUNT(*)
FROM ratings
WHERE book_id = r.book_id)<= 3
THEN r.rating END) AS остальные_книги
FROM ratings AS r'''
pd.io.sql.read_sql(query, con = engine)

,средняя_оценка,остальные_книги
0,3.938398,3.890909


 Выведите топ пять пользователей по суммарному показателю написанных ревью и поставленных оценок

In [14]:
query = ''' SELECT rev.username,
            COUNT(DISTINCT review_id) AS ревью,
            COUNT(DISTINCT rating_id) AS оценки,
            COUNT(DISTINCT review_id) + COUNT(DISTINCT rating_id) AS  итого
FROM reviews AS rev
FULL JOIN ratings ON rev.username = ratings.username
GROUP BY rev.username
ORDER BY итого DESC
LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)

,username,ревью,оценки,итого
0,sfitzgerald,28,55,83
1,martinadam,27,56,83
2,richard89,26,55,81
3,jennifermiller,25,53,78
4,paul88,22,56,78


### Общий вывод.

Итак, нам для анализа предоставили 5 таблиц : books, authors, publishers, ratings,  и reviews.
    
    По поставленным вопросам мы сделали следующие выводы:
        

1.Посчитайте, сколько книг вышло после 1 января 2000 года;

Добавилось целых 819 книг.

2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

У нас получилось 1000 книг ровно. Для каких-то книг сделали 7 обзоров, а для каких-то вообще нет. Оценка рейтинга гуляет, причем не видно на первый взгляд зависимости от наличия обзоров. Так у книги The Cat in the Hat and Other Dr. Seuss Favorites рейтинг 5, при отсутствии обзоров, а у книги Twilight (Twilight #1) рейтинг 3,66 при максимальных 7 обзорах. Вывод, сколько обзоров не делай, на рейтинг это не повляет, можно только привлечь пользователей для прочтения, а вот выводы они уже сами сделают. И оценка уже зависит только от автора книги и личных предпочтений читателя.

3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

Больше всего книг выпущено издательством Penguin Books - 42 книги. Хороший результат. Следующий Vintage с 31 книгой и замыкает тройку Grand Central Publishing с 25 книгами.

4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

Самая популярная это J.K. Rowling/Mary GrandPré с рейтингом 4,28. Не сильно отстал Markus Zusak/Cao Xuân Việt Khương с рейтингом 4,26. И очень рядом Джон Рональд Руэл Толкин с рейтингом 4,24.

5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

Пользователи, которые поставили больше 48 оценок в среднем пишут 24 обзора.
